In [1]:
import numpy as np
import ipyvolume as ipv
from IPython.core.debugger import set_trace
import ipywidgets as widgets

In [2]:
def Rx(theta):
    R = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return R

def Ry(theta):
    R = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return R

def Rz(theta):
    R = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0,0,1]
    ])
    return R
    

In [3]:
N=5
DH_alpha = np.array([            [0],                  [0],                 [-np.pi/2],              [-np.pi/2]       ])
DH_a =     np.array([            [15],                 [30],                [15],                    [0]              ])
DH_theta = np.array([[-np.pi/2],       [np.pi/2],              [np.pi/2],               [-np.pi/2],                [-np.pi/2]       ])
DH_d =     np.array([[0],              [0],                    [0],                     [0],                       [15]      ])


T_0_dash_0 = np.block([
    [Ry(np.pi/2), np.array([[0],[0],[0]])],
    [np.array([[0,0,0,1]])]
])

In [4]:
T_i__i_minus_1_dash=np.zeros([5,4,4])
T_i_dash_i= np.zeros([4,4,4])
T= np.zeros([5,4,4])

In [5]:
def T_update(DH_alpha, DH_a, DH_theta, DH_d):

    for i in np.array([1,2,3,4,5]):
        T_i__i_minus_1_dash[i-1,:,:] = np.block([
            [Rz(DH_theta[i-1]), np.array([[0],[0],[DH_d[i-1]]])],
            [np.array([[0,0,0,1]])]
        ])
    for i in np.array([1,2,3,4]):
        T_i_dash_i[i-1,:,:] = np.block([
            [Rx(DH_alpha[i-1]), np.array([[DH_a[i-1]],[0],[0]])],
            [np.array([[0,0,0,1]])]
        ])

    T[0,:,:] = T_0_dash_0@T_i__i_minus_1_dash[0,:,:]

    for ii in np.array([1,2,3,4]):
        T[ii,:,:]= T[ii-1,:,:]@T_i_dash_i[ii-1,:,:]@T_i__i_minus_1_dash[ii,:,:]


In [6]:
def link_graphics(z_r,z_l,z_u,x_r, x_l, x_u, N):

    #r=1
    #w=3
    #a=8
    #N=20

    theta = np.linspace(0, 2*np.pi, N, endpoint = True)
    Xa = np.array([0*np.cos(theta), z_r*np.cos(theta), z_r*np.cos(theta), 0*np.cos(theta)])
    Ya = np.array([0*np.sin(theta), z_r*np.sin(theta), z_r*np.sin(theta), 0*np.sin(theta)])
    Za = np.array([np.ones((N,))*z_l, np.ones((N,))*z_l, np.ones((N,))*z_u, np.ones((N,))*z_u]) 

    Yl = np.array([0*np.cos(theta), x_r*np.cos(theta), x_r*np.cos(theta), 0*np.cos(theta)])
    Zl = np.array([0*np.sin(theta), x_r*np.sin(theta), x_r*np.sin(theta), 0*np.sin(theta)])
    Xl = np.array([np.ones((N,))*0,np.ones((N,))*x_l,np.ones((N,))*x_u,np.ones((N,))*x_u]) 
    
    return [Xa, Ya, Za, Xl, Yl, Zl]


    

In [7]:
def link_graphics_transform(Xa, Ya, Za, Xl, Yl, Zl, T, plota, plotl):
    
    Ma = np.array([Xa, Ya, Za, Za*0+1])
    Ml = np.array([Xl, Yl, Zl, Zl*0+1])
    Ma_t = (np.matmul(T,np.moveaxis(Ma,0,1)))
    Ml_t = (np.matmul(T,np.moveaxis(Ml,0,1)))
    
    plota.x = Ma_t[:,0,:].reshape(-1)
    plota.y = Ma_t[:,1,:].reshape(-1)
    plota.z = Ma_t[:,2,:].reshape(-1)
    
    plotl.x = Ml_t[:,0,:].reshape(-1)
    plotl.y = Ml_t[:,1,:].reshape(-1)
    plotl.z = Ml_t[:,2,:].reshape(-1)
    

In [8]:
j = ipv.figure()
k_l=[1,2,3,4,5]
t_l=[1,2,3,4,5]
color_list = ["orange", "blue", "green", "yellow", "magenta"]

In [9]:
N_circle=20

Xa= np.zeros([5,4,N_circle])
Ya= np.zeros([5,4,N_circle])
Za= np.zeros([5,4,N_circle])
Xl= np.zeros([5,4,N_circle])
Yl= np.zeros([5,4,N_circle])
Zl= np.zeros([5,4,N_circle])



for ii in np.array([1,2,3,4,5]):
    if ii<5:
        [Xa[ii-1,:,:], Ya[ii-1,:,:], Za[ii-1,:,:], Xl[ii-1,:,:], Yl[ii-1,:,:], Zl[ii-1,:,:]] = link_graphics(1,-1.5, +1.5, 1, 0, DH_a[ii-1], N_circle)
    else:
        [Xa[ii-1,:,:], Ya[ii-1,:,:], Za[ii-1,:,:], Xl[ii-1,:,:], Yl[ii-1,:,:], Zl[ii-1,:,:]] = link_graphics(1,-15, 0,  4, -0.5, +0.5,  N_circle )
                
    k=ipv.plot_surface(Xa[ii-1,:,:], Ya[ii-1,:,:], Za[ii-1,:,:], color=color_list[ii-1])
    t=ipv.plot_surface(Xl[ii-1,:,:], Yl[ii-1,:,:], Zl[ii-1,:,:], color=color_list[ii-1])
    k_l[ii-1]=k
    t_l[ii-1]=t




In [10]:
def points_update():
    for ii in np.array([1,2,3,4,5]):
        link_graphics_transform(Xa[ii-1,:,:], Ya[ii-1,:,:], Za[ii-1,:,:], Xl[ii-1,:,:], Yl[ii-1,:,:], Zl[ii-1,:,:], T[ii-1,:,:], k_l[ii-1], t_l[ii-1])


In [11]:
def update(*args):
    DH_theta=np.array([
    theta1.value*np.pi/180,
    theta2.value*np.pi/180,
    theta3.value*np.pi/180,
    theta4.value*np.pi/180,
    theta5.value*np.pi/180,
    ])
    T_update(DH_alpha, DH_a, DH_theta, DH_d)
    points_update()



In [12]:
theta1 = widgets.FloatSlider(
    value=-90,
    min=-360,
    max=+360,
    step=0.1,
    description='First Joint rotation (Degrees):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='500px'),
    readout=True,
    readout_format='.1f',
)

theta1.observe(update, 'value')
                                
theta2 = widgets.FloatSlider(
    value=90,
    min=-360,
    max=+360,
    step=0.1,
    description='Second Joint rotation (Degrees):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='500px'),
    readout=True,
    readout_format='.1f',
)

theta2.observe(update, 'value')

theta3 = widgets.FloatSlider(
    value=90,
    min=-360,
    max=+360,
    step=0.1,
    description='Third Joint rotation (Degrees):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='500px'),
    readout=True,
    readout_format='.1f',
)

theta3.observe(update, 'value')

theta4 = widgets.FloatSlider(
    value=-90,
    min=-360,
    max=+360,
    step=0.1,
    description='Fourth Joint rotation (Degrees):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='500px'),
    readout=True,
    readout_format='.1f',
)

theta4.observe(update, 'value')

theta5 = widgets.FloatSlider(
    value=90,
    min=-360,
    max=+360,
    step=0.1,
    description='Fifth Joint rotation (Degrees):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    layout = widgets.Layout(width='500px'),
    readout=True,
    readout_format='.1f',
)

theta5.observe(update, 'value')

In [13]:
ipv.pylab.xlim(-10, 10)
ipv.pylab.ylim(-10, 10)
ipv.pylab.zlim(-10, 10)

#ipv.show()

In [14]:
widgets.VBox([  j,
              theta1,
              theta2,
              theta3,
              theta4,
              theta5
             ] )